## Import packages

In [ ]:
import optuna
import mlflow
from  statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## Mlflow settings

In [ ]:
mlflow.set_tracking_uri("http://localhost:5007")

## Load data

In [ ]:
data = pd.read_parquet("../data/processed/gold_macro_combined.parquet")
data

## Prepare data for modelling

In [ ]:
# Ensure the index is a Timestamp
data.index = pd.to_datetime(data.index)

# Fill missing values in continuous numerical columns
data['Gold_ETFs'] = data['Gold_ETFs'].fillna(data['Gold_ETFs'].mean())
data['GPRD'] = data['GPRD'].fillna(data['GPRD'].mean())

data = data.rename(columns={'Gold': 'gold_price'})

# Trim data to where Bitcoin price starts (from 17 September 2014)
df = data[data.index >= '2014-09-17']
#df = df[df.index <= '2025-06-09']

# Check remaining missing values
missing = df.isnull().sum()
print(missing[missing > 0])

### Stationarity Transformation (Differencing if needed)

In [ ]:
def adf_test(series):
    """Returns True if series is stationary based on p-value < 0.05"""
    result = adfuller(series.dropna(), autolag='AIC')
    return result[1] < 0.05

def make_stationary(df):
    df_stationary = df.copy()
    transformed_features = []
    
    for feature in df.columns:
        print(f"Testing feature: {feature}")
        if adf_test(df[feature]):
            print(f" -> {feature} is stationary.")
        else:
            print(f" -> {feature} is NOT stationary. Applying differencing...")
            df_stationary[feature] = df[feature].diff()
            transformed_features.append(feature)
            
            if adf_test(df_stationary[feature]):
                print(f" -> {feature} is stationary after differencing.")
            else:
                print(f" -> {feature} is STILL non-stationary after differencing.")
    
    df_stationary = df_stationary.dropna()

    print("\nSummary:")
    print(f"Features differenced: {transformed_features}")
    
    return df_stationary

df_stationary = make_stationary(df)

### Feature Reduction Based on Correlation

In [ ]:
# Check correlation matrix before feature removal
plt.figure(figsize=(20,25))
sns.heatmap(df.corr(), annot=True, fmt='.2f', cmap='coolwarm', annot_kws={"size": 14},
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.5})
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.title('Feature Correlation Heatmap (Before Feature Removal)', fontsize=20)
plt.tight_layout()
plt.show()

###  Drop highly correlated features

In [ ]:
df = df.drop(['Unemployment_Rate', 'EUR_USD', 'Vix', 'GPD',
              'M2_Euro', 'US_Interest_Rate', 'Zinsen', 'CPI', 'Bitcoin'], axis=1)
df = df_stationary
df['lag_1'] = df['gold_price'].shift(1)
df['lag_7'] = df['gold_price'].shift(7)
df['lag_14'] = df['gold_price'].shift(14)
df = df.dropna()
df

## ARIMAX Modelling

### ARIMAX Hyperparams optimization

In [ ]:
df_orig = df.copy()
# Define data
#df_orig = df_orig.rename(columns={'Gold': 'gold_price'})
target_col = 'gold_price'
exog_cols = [col for col in df_orig.columns if col != target_col]

df_orig.index = pd.to_datetime(df_orig.index)
df_orig = df_orig.sort_index()

y_full = df_orig[target_col]
X_full = df_orig[exog_cols]

# Time series split
tscv = TimeSeriesSplit(n_splits=5)

def objective(trial):
    # SARIMAX params
    p = trial.suggest_int('p', 0, 3)
    d = trial.suggest_int('d', 0, 1)
    q = trial.suggest_int('q', 0, 3)
    #P = trial.suggest_int('P', 0, 1)
    #D = trial.suggest_int('D', 0, 1)
    #Q = trial.suggest_int('Q', 0, 1)
    #s = trial.suggest_categorical('seasonal_period', [0, 7, 12])  # No seasonality, weekly, or monthly

    rmses = []

    with mlflow.start_run(nested=True):
        mlflow.log_params({
            'p': p, 'd': d, 'q': q,
            #'P': P, 'D': D, 'Q': Q,
            #'seasonal_period': s,
            'exogenous_features': ','.join(exog_cols)
        })

        for fold, (train_idx, test_idx) in enumerate(tscv.split(y_full)):
            y_train, y_test = y_full.iloc[train_idx], y_full.iloc[test_idx]
            exog_train, exog_test = X_full.iloc[train_idx], X_full.iloc[test_idx]

            try:
                model = ARIMA(
                    y_train,
                    exog=exog_train,
                    order=(p, d, q),
                    #seasonal_order=(P, D, Q, s),
                    enforce_stationarity=True,
                    enforce_invertibility=True
                )
                results = model.fit(disp=False)

                forecast = results.forecast(steps=len(y_test), exog=exog_test)
                forecast = pd.Series(forecast, index=y_test.index)

                rmse = np.sqrt(mean_squared_error(y_test, forecast))

                rmses.append(rmse)

                mlflow.log_metric(f"rmse_fold_{fold+1}", rmse)

            except Exception as e:
                mlflow.log_param("error", str(e))
                return float("inf")

        avg_rmse = np.mean(rmses)

        mlflow.log_metric("avg_rmse", avg_rmse)

        return avg_rmse


# Run study
mlflow.set_experiment("Gold Price Forecasting - ARIMAX Model - With Features + lags + correlation feature removal")

with mlflow.start_run(run_name="optuna_arimax_study"):
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100, timeout=900)

    best_trial = study.best_trial
    mlflow.log_params(best_trial.params)
    mlflow.log_metric("best_avg_rmse", best_trial.value)

print("Best trial:")
print("  RMSE:", best_trial.value)
print("  Params:", best_trial.params)


### Forecast Gold Price using ARIMAX

In [ ]:
#df = df.rename(columns={'Gold': 'gold_price'})
target_col = 'gold_price'
exog_cols = [col for col in df.columns if col != target_col]

df.index = pd.to_datetime(df.index)
df = df.sort_index()
print("df:")
print(df)

future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=14, freq='D')
delta_df = df[exog_cols].diff().iloc[-7:]
avg_trend = delta_df.mean()
base_std = delta_df.std()
last_row = df[exog_cols].iloc[-1]

def generate_X_future(noise_multiplier=0.5, seed=42):
    np.random.seed(seed)
    noise_std = base_std * noise_multiplier
    X_future = pd.DataFrame(index=future_dates, columns=exog_cols)
    for i, date in enumerate(future_dates):
        trend_value = last_row + (i + 1) * avg_trend
        noise = np.random.normal(loc=0, scale=noise_std.values)
        X_future.loc[date] = trend_value + noise
    return X_future.astype(float)

X_future_high_noise = generate_X_future(noise_multiplier=1.0)

y_full = df[target_col]
X_full = df[exog_cols]
mlflow.set_experiment("ARIMA_Forecast_Deployment With Features + lags + correlation feature removal")

with mlflow.start_run(run_name="ARIMA_14d_HighNoise + Features + lags + correlation feature removal"):
    model = ARIMA(
        y_full,
        exog=X_full,
        order=(best_trial.params["p"], 0, best_trial.params["q"]),
        #seasonal_order=(best_trial.params["P"], best_trial.params["D"], best_trial.params["Q"], best_trial.params["seasonal_period"]),
        enforce_stationarity=True,
        enforce_invertibility=True
    )
    results = model.fit()

    # Log parameters and metrics
    mlflow.log_params({
        "order": f"({best_trial.params['p']}, {best_trial.params['d']}, {best_trial.params['q']})",
        #"seasonal_order": f"({best_trial.params['P']}, {best_trial.params['D']}, {best_trial.params['Q']}, {best_trial.params['seasonal_period']})",
        "noise_multiplier": 1.0
    })
    mlflow.log_metrics({
        "aic": results.aic,
        "bic": results.bic
    })

    # Forecasting
    forecast_14d = results.forecast(steps=14, exog=X_future_high_noise)
    forecast_14d.index = future_dates
    
    last_observed_value = data[target_col].iloc[-1]
    forecast_14d = forecast_14d.cumsum() + last_observed_value

    print("last_observed_value:", last_observed_value)

    print(forecast_14d)
    # Save forecast
    forecast_14d = forecast_14d.reset_index()
    forecast_14d.columns = ["date", "forecast"]
    forecast_csv_path = "forecast_14d.csv"
    forecast_14d.to_csv(forecast_csv_path, index=False)
    mlflow.log_artifact(forecast_csv_path)

    # Plot results
    filtered_df = data[data.index >= '2025-06-10']
    print(filtered_df)
    plt.figure(figsize=(14, 6))
    plt.plot(filtered_df.index, filtered_df['gold_price'], label='Historical Gold Price')
    plt.plot(forecast_14d['date'], forecast_14d['forecast'], label='Forecasted Gold Price (Next 14 Days)', linestyle='--')
    plt.title("Gold Price Forecast - Next 14 Days")
    plt.xlabel("Date")
    plt.ylabel("Gold Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plot_path = "forecast_plot.png"
    plt.savefig(plot_path)
    mlflow.log_artifact(plot_path)
    #plt.close()
    plt.show()

### Forecast the last 14 days

In [ ]:
df_forecast = df.copy()

forecast_horizon = 14
df_forecast = df_forecast.sort_index()

y_full = df_forecast[target_col]
X_full = df_forecast[exog_cols]

y_train = y_full.iloc[:-forecast_horizon]
X_train = X_full.iloc[:-forecast_horizon]

y_test = y_full.iloc[-forecast_horizon:]
X_test = X_full.iloc[-forecast_horizon:]
test_dates = y_test.index

mlflow.set_experiment("ARIMAX_Evaluation - With Features + lags + correlation feature removal")

with mlflow.start_run(run_name="ARIMAX_Last14d_ActualExog - With Features + lags + correlation feature removal") as run:
    model = ARIMA(
        y_train,
        exog=X_train,
        order=(best_trial.params["p"], 0, best_trial.params["q"]),
        #seasonal_order=(best_trial.params["P"], best_trial.params["D"], best_trial.params["Q"], best_trial.params["seasonal_period"]),
        enforce_stationarity=True,
        enforce_invertibility=True
    )
    results = model.fit()

    # Forecast next 14 days using actual exogenous test data
    forecast = results.forecast(steps=forecast_horizon, exog=X_test)
    forecast.index = test_dates
        
    last_observed_value = data[target_col].iloc[-1]
    forecast = forecast.cumsum() + last_observed_value
    y_test = y_test.cumsum() + last_observed_value
    print("forecast:")
    print(forecast)
    # Evaluation 
    rmse = np.sqrt(mean_squared_error(y_test, forecast))
    print("RMSE on last 14 days:", rmse)

    # Log parameters and RMSE
    mlflow.log_params({
        "order": f"({best_trial.params['p']}, {best_trial.params['d']}, {best_trial.params['q']})",
        #"seasonal_order": f"({best_trial.params['P']}, {best_trial.params['D']}, {best_trial.params['Q']}, {best_trial.params['seasonal_period']})"
    })
    mlflow.log_metrics({
        "aic": results.aic,
        "bic": results.bic,
        "rmse": rmse
    })

    # Plotting
    plt.figure(figsize=(14, 6))
    plt.plot(test_dates, y_test, label="True Gold Price", marker='o')
    plt.plot(test_dates, forecast, label="Forecast (Last 14 Days)", linestyle='--', marker='x')
    plt.title("ARIMAX Forecast - Last 14 Days")
    plt.xlabel("Date")
    plt.ylabel("Gold Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plot_path = "last_14d_forecast_plot.png"
    plt.savefig(plot_path)
    mlflow.log_artifact(plot_path)
    plt.close()
